### Imports and Setup

In [ ]:
import odrive
import time
from IPython.display import display
import ipywidgets as widgets

odrv0 = odrive.find_any()
axis0 = odrv0.axis0
axis1 = odrv0.axis1

inbound_pin = 8
outbound_pin = 9

def getPin(pin):
    mask = 1 << (pin - 1)
    return (odrv0.get_gpio_states() & mask) != 0


### Error Checking

In [ ]:
odrv0.clear_errors()

In [ ]:
odrive.utils.dump_errors(odrv0)

### Full State Calibration

In [ ]:
axis1.requested_state = odrive.enums.AXIS_STATE_FULL_CALIBRATION_SEQUENCE

while axis1.current_state != 1:
    print(f"Current State: {axis1.current_state}", end="\r")
    time.sleep(0.1)
print("\nDone!")


In [ ]:
axis1.motor.config.pre_calibrated = True
axis1.encoder.config.pre_calibrated = True
odrv0.save_configuration()

### Index Search

In [ ]:

axis1.requested_state = odrive.enums.AXIS_STATE_ENCODER_INDEX_SEARCH

while axis1.current_state != 1:
    time.sleep(0.1)

print("Done!")


### Encoder Logging

In [ ]:
start = axis1.encoder.shadow_count
while True:
    print(axis1.encoder.shadow_count-start, end="\r")
    time.sleep(0.01)

### Velocity Control

In [ ]:
up_button = widgets.Button(description='Up', layout=widgets.Layout(width='100px',height="100px"))
down_button = widgets.Button(description='Down', layout=widgets.Layout(width='100px',height="100px"))
stop_button = widgets.Button(description='Stop', layout=widgets.Layout(width='100px',height="100px"))
change_dir_button = widgets.Button(description='Change Dir', layout=widgets.Layout(width='100px',height="100px"))

counter = widgets.IntText(value=0, description='Vel:')

def onUpButton(button):
    counter.value += 1
    axis1.requested_state = 8
    axis1.controller.input_vel = counter.value


def onDownButton(button):
    counter.value -= 1
    axis1.requested_state = 8
    axis1.controller.input_vel = counter.value

def onStopButton(button):
    axis1.controller.input_vel = 0
    counter.value = 0
    axis1.requested_state = 1

def onChangeDir(button):
    counter.value *= -1
    axis1.requested_state = 8
    axis1.controller.input_vel = counter.value


up_button.on_click(onUpButton)
down_button.on_click(onDownButton)
stop_button.on_click(onStopButton)
change_dir_button.on_click(onChangeDir)

box_layout = widgets.Layout(display='flex', flex_flow='row', align_items='center')

container = widgets.Box(children=[up_button, down_button, change_dir_button, stop_button, counter], layout=box_layout)

display(container)


In [ ]:
axis1.requested_state = 1

### Estops

In [ ]:
while True:
    print(f"inbound: {getPin(inbound_pin)}, outbound: {getPin(outbound_pin)}", end="\r")
    time.sleep(0.01)

In [ ]:
import odrive
import re
errors = {}
for name, value in odrive.enums.__dict__.items():
    if not callable(value) and not name.startswith('__') and re.match(".*_ERROR_.*", name):
        errors[name] = value
print(sorted(errors))
